In [116]:
# This code will create a table for each chromosome showing the
# normalized read depth across teh chromosome annoated back to the region

# Step 1: Check working directory
import os
os.chdir('/Users/evaedwards/Final-Year-Project/Datasets/TXT')
print(os.getcwd())

# Step 2: Make txt. file for the chromosome i.e CP034462
import pandas as pd
import glob

# Define input and output file pathsfrom Bio import SeqIO
import pandas as pd

# File paths
genbank_file = "GCA_004217705.1_ASM421770v1_genomic-1.gbk"
record_id = "CP034462.1"  # Change this to the appropriate chromosome ID

# Parse the GenBank file and find the correct record
record = None
for seq_record in SeqIO.parse(genbank_file, "genbank"):
    if seq_record.id == record_id:
        record = seq_record
        break

if record is None:
    raise ValueError(f"Record ID '{record_id}' not found in {genbank_file}")

# Initialize lists to store genic and intergenic region data
genic_regions = []
intergenic_regions = []

# Extract genic regions (prefer 'gene' over 'CDS')
seen_genes = set()
for feature in record.features:
    if feature.type in ["gene", "CDS"]:  # Adjust as needed
        start = feature.location.start
        end = feature.location.end
        gene_name = feature.qualifiers.get("gene", ["Unknown"])[0]  # Get gene name if available
        product = feature.qualifiers.get("product", ["unknown"])[0]
        
        # Avoid duplicates by using (start, end) as a unique key
        if (start, end) not in seen_genes:
            seen_genes.add((start, end))
            genic_regions.append((start, end, gene_name, product))

# Sort genic regions by start position
genic_regions.sort()

# Extract intergenic regions
prev_end = 0  # Start of chromosome
intergenic_names = []

for i in range(len(genic_regions)):
    start, end, gene_name, _ = genic_regions[i]

    # If there's a gap, define an intergenic region
    if prev_end < start:
        # Get flanking gene names
        upstream_gene = genic_regions[i - 1][2] if i > 0 else "-"
        downstream_gene = gene_name
        intergenic_name = f"{upstream_gene} / {downstream_gene}"
        intergenic_regions.append((prev_end, start, start - prev_end, intergenic_name))
    
    prev_end = end  # Move to the end of current gene

# Capture last intergenic region (if any)
if prev_end < len(record.seq):
    last_gene = genic_regions[-1][2] if genic_regions else "-"
    intergenic_name = f"{last_gene} / -"
    intergenic_regions.append((prev_end, len(record.seq), len(record.seq) - prev_end, intergenic_name))

# Convert to pandas DataFrame
genic_df = pd.DataFrame(genic_regions, columns=["Start", "End", "Gene", "Product"])
intergenic_df = pd.DataFrame(intergenic_regions, columns=["Start", "End", "Length", "Gene"])

# Add a column to differentiate genic vs. intergenic
genic_df["Type"] = "genic"
intergenic_df["Type"] = "intergenic"

# Merge both DataFrames
combined_df = pd.concat([genic_df, intergenic_df], ignore_index=True)

# Sort by genomic position
combined_df = combined_df.sort_values(by="Start").reset_index(drop=True)

# Save to CSV
combined_df.to_csv("combined_regions62.csv", index=False)

print("Merged CSV file saved as 'combined_regions62.csv'")

input_files = glob.glob("TXT/*.txt")  # Adjust to match your directory
output_folder = "CP034462_txt/"  # Folder to store filtered files
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Process each file
for file in input_files:
    strain_name = os.path.basename(file).replace(".txt", "")  # Extract strain name
    
    # Load the coverage data (assuming tab-separated format, no headers)
    df = pd.read_csv(file, sep="\t", header=None, names=["Chromosome", "Position", "Coverage"])
    
    # Filter only for the chromosome i.e CP034462
    df_filtered = df[(df["Chromosome"] == "CP034462")]
    
    # Save the filtered data - change name for the chromosome i.e CP034462
    output_file = os.path.join(output_folder, f"{strain_name.upper()}_CH62.txt")
    df_filtered.to_csv(output_file, sep="\t", index=False, header=False)
    
    print(f"Saved filtered data for {strain_name} to {output_file}")



/Users/evaedwards/Final-Year-Project/Datasets/TXT
Merged CSV file saved as 'combined_regions61.csv'
Saved filtered data for matc2 to CP034461_txt/MATC2_CH61.txt
Saved filtered data for WS4 to CP034461_txt/WS4_CH61.txt
Saved filtered data for jog19 to CP034461_txt/JOG19_CH61.txt
Saved filtered data for WS5 to CP034461_txt/WS5_CH61.txt
Saved filtered data for matc3 to CP034461_txt/MATC3_CH61.txt
Saved filtered data for matc1 to CP034461_txt/MATC1_CH61.txt
Saved filtered data for WS7 to CP034461_txt/WS7_CH61.txt
Saved filtered data for WS6 to CP034461_txt/WS6_CH61.txt
Saved filtered data for matc4 to CP034461_txt/MATC4_CH61.txt
Saved filtered data for WS2 to CP034461_txt/WS2_CH61.txt
Saved filtered data for WS3 to CP034461_txt/WS3_CH61.txt
Saved filtered data for WS1 to CP034461_txt/WS1_CH61.txt
Saved filtered data for jog20 to CP034461_txt/JOG20_CH61.txt
Saved filtered data for jog21 to CP034461_txt/JOG21_CH61.txt
Saved filtered data for mat22 to CP034461_txt/MAT22_CH61.txt
Saved filtere

In [122]:
from Bio import SeqIO
import pandas as pd

# File paths
genbank_file = "GCA_004217705.1_ASM421770v1_genomic-1.gbk"
record_id = "CP034462.1"  # Change this to the appropriate chromosome ID

# Initialize lists to store genic and intergenic region data
genic_regions = []
intergenic_regions = []

# Open the GenBank file
with open(genbank_file, "r") as handle:
    for record in SeqIO.parse(handle, "genbank"):
        if record.id == record_id:
            seen_genes = set()

            # Extract genic regions (specifically for CDS and gene features)
            for feature in record.features:
                if feature.type == "CDS":  # Focus on coding sequences
                    start = feature.location.start
                    end = feature.location.end
                    
                    # Get gene name and product (or 'unknown' if not available)
                    gene_name = feature.qualifiers.get("gene", ["unknown"])[0]
                    product = feature.qualifiers.get("product", ["unknown"])[0]
                    
                    # Avoid duplicates by using (start, end) as a unique key
                    if (start, end) not in seen_genes:
                        seen_genes.add((start, end))
                        genic_regions.append((start, end, gene_name, product))

            # Sort genic regions by start position
            genic_regions.sort()

            # Extract intergenic regions
            prev_end = 0  # Start of chromosome
            for i in range(len(genic_regions)):
                start, end, gene_name, product = genic_regions[i]

                # If there's a gap, define an intergenic region
                if prev_end < start:
                    # Get flanking gene names and product
                    upstream_gene = genic_regions[i - 1][2] if i > 0 else "-"
                    downstream_gene = gene_name
                    upstream_product = genic_regions[i - 1][3] if i > 0 else "-"
                    downstream_product = product
                    intergenic_name = f"{upstream_gene} / {downstream_gene}"
                    intergenic_product = f"{upstream_product} / {downstream_product}"

                    intergenic_regions.append((prev_end, start, start - prev_end, intergenic_name, intergenic_product))
                
                prev_end = end  # Move to the end of current gene

            # Capture last intergenic region (if any)
            if prev_end < len(record.seq):
                last_gene = genic_regions[-1][2] if genic_regions else "-"
                last_product = genic_regions[-1][3] if genic_regions else "-"
                intergenic_name = f"{last_gene} / -"
                intergenic_product = f"{last_product} / -"
                intergenic_regions.append((prev_end, len(record.seq), len(record.seq) - prev_end, intergenic_name, intergenic_product))

# Convert to pandas DataFrame
genic_df = pd.DataFrame(genic_regions, columns=["Start", "End", "Gene", "Product"])
intergenic_df = pd.DataFrame(intergenic_regions, columns=["Start", "End", "Length", "Gene", "Product"])

# Add a column to differentiate genic vs. intergenic
genic_df["Type"] = "genic"
intergenic_df["Type"] = "intergenic"

# Merge both DataFrames
combined_df = pd.concat([genic_df, intergenic_df], ignore_index=True)

# Sort by genomic position
combined_df = combined_df.sort_values(by="Start").reset_index(drop=True)

# Save to CSV
combined_df.to_csv("combined_regions_with_product62.csv", index=False)

print("Merged CSV file saved as 'combined_regions_with_product62.csv'")


Merged CSV file saved as 'combined_regions_with_product62.csv'


In [124]:
# Merge with txt data

# Step 4: 

import os
import glob
import pandas as pd
import numpy as np

print(os.getcwd())

# Define paths
base_dir = "/Users/evaedwards/Final-Year-Project/Datasets/TXT"
annotated_file = os.path.join(base_dir, "combined_regions_with_product62.csv")
coverage_files = glob.glob(os.path.join(base_dir, "CP034462_txt", "*.txt"))
wg_median_file = os.path.join(base_dir, "WG_Median_Coverage_CAPITALIZED.csv")
apc_median_file = os.path.join(base_dir, "WG_Median_Coverage_APC.csv")  # Added APC median coverage file

# Load annotated genome data
genes_df = pd.read_csv(annotated_file)

# Load whole-genome median coverage data
wg_median_df = pd.read_csv(wg_median_file)
wg_median_dict = dict(zip(wg_median_df["Strain"], wg_median_df["Median Coverage"]))  # {Strain: MedianCoverage}

# Load APC median coverage data
apc_median_df = pd.read_csv(apc_median_file)
wg_median_dict.update(dict(zip(apc_median_df["Strain"], apc_median_df["Median Coverage"])))  # Merge APC data

# Store processed data
final_data = []

# Process each strain's coverage file
for file in coverage_files:
    strain_name = os.path.basename(file).split("_")[0].upper()  # Extract strain name
    
    # Load coverage data
    coverage_df = pd.read_csv(file, sep="\t", header=None, names=["Chromosome", "Position", "Coverage"])
    coverage_dict = dict(zip(coverage_df["Position"], coverage_df["Coverage"]))  # {Position: Coverage}
    
    # Get the whole-genome median for this strain (default to 1 if missing to avoid division errors)
    wg_median = wg_median_dict.get(strain_name, 1)
    
    # Process each gene
    for _, gene in genes_df.iterrows():
        gene_function = gene["Product"]
        gene_id = gene["Gene"]
        start_pos, end_pos = gene["Start"], gene["End"]
        length = end_pos - start_pos  

        # Extract coverage values within the gene range
        coverage_values = [coverage_dict[pos] for pos in range(start_pos, end_pos + 1) if pos in coverage_dict]

        # Calculate median coverage for the gene
        median_coverage = round(np.median(coverage_values)) if coverage_values else None

        # Calculate scaled coverage
        scaled_coverage = round(median_coverage / wg_median, 3) if median_coverage is not None else None

        # Store result
        final_data.append([gene_id, gene_function, start_pos, end_pos, length, strain_name, scaled_coverage])

# Convert to DataFrame
final_df = pd.DataFrame(final_data, columns=["Gene ID", "Gene Function", "Start Position", "End Position", "Length", "Strain", "Scaled Coverage"])

# Pivot table to make strains as columns
final_pivot = final_df.pivot(index=["Gene ID", "Gene Function", "Start Position", "End Position", "Length"], columns="Strain", values="Scaled Coverage")

# Reset column names for clarity
final_pivot.columns = [f"{strain}" for strain in final_pivot.columns]
final_pivot.reset_index(inplace=True)

# Save the final table
output_file = os.path.join(base_dir, "Table_CRC_allregions62.csv")
final_pivot.to_csv(output_file, index=False)

# Display result
print(final_pivot.head())

/Users/evaedwards/Final-Year-Project/Datasets/TXT
                     Gene ID  \
0            - / MPUL0G00100   
1                MPUL0G00100   
2  MPUL0G00100 / MPUL0G00110   
3                MPUL0G00110   
4  MPUL0G00110 / MPUL0G00120   

                                       Gene Function  Start Position  \
0  - / Transcriptional activator of glycolytic en...               0   
1    Transcriptional activator of glycolytic enzymes            6215   
2  Transcriptional activator of glycolytic enzyme...            8981   
3    Transcriptional activator of glycolytic enzymes           10919   
4  Transcriptional activator of glycolytic enzyme...           13685   

   End Position  Length   APC12    JOG1  JOG10  JOG11  JOG12  ...    WS11  \
0          6215    6215   9.746   9.758  6.883  6.556  6.905  ...   9.707   
1          8981    2766  10.034  10.159  7.255  6.894  7.267  ...  10.445   
2         10919    1938  10.203  10.176  7.173  6.908  7.155  ...  10.183   
3         13685 